In [ ]:
MaxEntに適用するための出現データセット作成，およびMaxEnt後にハビタットマップを作るまでのチュートリアル

In [1]:
#import libraries
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import os
import cv2
import habmaptools

各種の出現データと地形条件との関係を調べるまでのチュートリアル

In [3]:
#出現データの取得
path_output=os.getcwd()+"/sample/sample_output/"
species=["Hroretzi", "Btrigonus"]
species_2=["Hroretzi_1811", "Btrigonus_1811", "Btrigonus_1908"]

path_terrain_variables=os.getcwd()+"/sample/sample_output/terrain_variables_modelA01.csv"
path_occurrence_dataset=os.getcwd()+"/sample/sample_dataset/"
#speciesは，Species_yymm.plyのような形式で保存されています．
#異なる年月のデータを区別しない場合にはSpeciesのみ，区別する場合にはspecies_yymmでspeciesを指定してください．
#与えたspeceisには
#backgroundデータは非常に大量になるため，すべてのデータを用いず一部のデータのみを用いることを推奨します．
#sep=9だと9個おきにデータを取得してbackgroundとすることを意味します．
habmaptools.make_master_occurrence(path_terrain_variables, path_occurrence_dataset, species, path_output, sep=10)
pd.read_csv(path_output+"/master_occurrence.csv")

OSError: Initializing from file failed

重要な地形条件を調べるために，MaxEntを実施するまでのチュートリアル

In [ ]:
def main_analysis(path_now, path_out2, path_dataset, model_file_train, model_file_test, asc, species, param_all_raw):
    import numpy as np
    import pandas as pd
    import os
    mesh_nums=[0]
    for model_index in model_file_train+model_file_test:
        faces_data=np.array(pd.read_csv(path_now+r"Sfm_model/model"+model_index+"/ply_parts/faces_dataset.csv", header=None))
        mesh_nums.append(len(faces_data))

    path_out=path_now+"/maxent_data/"+path_out2
    os.makedirs(path_out, exist_ok=True)
    
    model_file_all=model_file_train+model_file_test

    #地形条件のデータセット(raw)
    make_env_dataset2(param_all_raw, model_file_all, path_now+"Sfm_model/", path_out, mode="raw_add", filename="_raw")
    #地形条件のデータセット(mesh)
    make_env_dataset2(param_all_raw, model_file_all, path_now+"Sfm_model/", path_out, mode="mesh_add", filename="_mesh")

    #出現データセット
    make_occurrence_dataset2(species, model_file_train, path_now+"Sfm_model/", path_out, path_dataset, filename="")

    #種＆地形条件のデータセット
    path_env=path_out+"/envdata_raw.csv"
    path_occurrence=path_out+"/occurrence.csv"
    make_master_occurrence(path_env, path_occurrence, species, path_out)

    #maxent用データの準備(ascファイルと出現データをmaxent_data/**/.asc階層に)
    if(asc==True):
        env_data_all=pd.read_csv(path_out+r"/envdata_raw.csv")
        for (i, param_name) in enumerate(param_all_raw):
            os.makedirs(path_out+"/asc_file/", exist_ok=True)
            param_to_asc(param_name, env_data_all.iloc[:,i], path_out+"/asc_file/")
    else:
        print("asc file not renewed.")

    s="train: "+str(model_file_train)+"test: "+str(model_file_test)
    path_w=path_out+"/use_model.txt"
    with open(path_w, mode='w') as f:
        f.write(s)